MultiIndex is an index that allows for multiple levels/layer/tiers. Effectively the way that Pandas allows us to have multi-dimensional datasets. In the past, we can reference any given row by a single index label. When it comes to muti-indexes, we will have to index each row with a combination of labels in sequence. 

Some additional concepts will be explored including moving index labels from columns to rows and vice-versa, creating pivot tables to have aggregate summaries of our data as well as melting data (opposite of pivoting data). 

Topics covered in this section:
- Create a MultiIndex on a DataFrame with the .set_index() method. 
- Extract index level values with the .get_level_values() method. 
- Change index level name with the .set_names() method. 
- The .sort_index() method on a MultiIndex DataFrame.
- Extract rows from a MultiIndex DataFrame.
- The .transpose() method on a MultiIndex DataFrame.
 - The .swaplevel() method. 
 - The .stack() method. 
 - The .unstack() method. 
 - The .pivot() method. 
 - Use the .pivot_table() method to create an aggregate summary of a DataFrame. 
 - Use the pd.melt() method to create a narrow dataset from a wide one. 

In [2]:
import pandas as pd
pd.__version__

'1.1.3'

In [4]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"])
# bm["Date"] = pd.to_datetime(bm["Date"])    Another way to change dtype of Date column to DATETIME format. 

bm.dtypes
bm.info() # Info on the data structure of the DataFrame. 
bm

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 652 non-null    datetime64[ns]
 1   Country              652 non-null    object        
 2   Price in US Dollars  652 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 15.4+ KB


,Date,Country,Price in US Dollars
0,2016-01-01,Argentina,2.39
1,2016-01-01,Australia,3.74
2,2016-01-01,Brazil,3.35
3,2016-01-01,Britain,4.22
4,2016-01-01,Canada,4.14
...,...,...,...
647,2010-01-01,Turkey,3.83
648,2010-01-01,UAE,2.99
649,2010-01-01,Ukraine,1.83
650,2010-01-01,United States,3.58


Create a MultiIndex with the .set_index() Method

In [5]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"])
bm.head()

,Date,Country,Price in US Dollars
0,2016-01-01,Argentina,2.39
1,2016-01-01,Australia,3.74
2,2016-01-01,Brazil,3.35
3,2016-01-01,Britain,4.22
4,2016-01-01,Canada,4.14


In [6]:
# .set_index() : previously for the keys parameter, we only input one column name to set as the index label. 
# Can pass a list as an argument for the keys parameter to set have multiple layer for the index. 
bm.nunique()
bm.set_index(keys = ["Date", "Country"], inplace = True)

# Now we have a MultiIndex, and a DataFrane with a single column (Price in US Dollars). 
# Two layers for the multiple: the outer layer (Date) and the internal layer (Country). 
# A single date emcompasses all the internal nested values within Countries. The order of the keys will determine the order of the 
# levels in the MultiIndex. There is no limit to the number of levels for the MultiIndex. 

# General practice is to have the column with the least number of unique values as the outer layer (first column in the list). 

In [24]:
bm.head()

Price in US Dollars
Date       Country                       
2016-01-01 Argentina                 2.39
           Australia                 3.74
           Brazil                    3.35
           Britain                   4.22
           Canada                    4.14

In [25]:
bm.sort_index(inplace = True)
bm.head(3)

# All the layers will be sorted, starting from the outer layer (Date) and then within each date, the second layer (Country) will be sorted.
# In ascending order by default. Can set ascending = False. 

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76

In [9]:
bm.index

# Gives the underlying attribute that is storing the index. We obtain a MultiIndex object. Think of each comination of levels in a muti-level index 
# as a tuple of two values (since there are two indexes). These are the unique identifier of each row. 
# Both indexes will be required in order to acces a value. 

bm.index.names # Returns the names of the indices, in addition, each layer is given an index position. 
type(bm.index) 
bm.index[0] # Shows the combination of indices that allows us to access the very first row. 

(Timestamp('2016-01-01 00:00:00'), 'Argentina')

Extract Index Level Values with the .get_level_values() Method 

This method is invoked on the index of the MultiIndex DataFrame, it extracts all of the values from a level of the MultiIndex. 

In [11]:
# Alternate way to set the MultiIndex is to set it when importing the DataFrame. 
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"], index_col = ["Date", "Country"])
bm.head()

Price in US Dollars
Date       Country                       
2016-01-01 Argentina                 2.39
           Australia                 3.74
           Brazil                    3.35
           Britain                   4.22
           Canada                    4.14

In [20]:
# .get_level_value() : is operated on the bm.index

datetime_index = bm.index.get_level_values("Date") # Targets the values in the Date level index (DATETIME format). 
datetime_index

# bm.index.get_level_values(0) # We can use the index position too. 

# Returns the the values for the specified index level. 

# Under the hood of the MultiIndex contains the differet index objects within it. 
# Since we changed the Dates column to DateTime format when we imported the DataFrame, the Date level index is a DateTime index. 

DatetimeIndex(['2016-01-01', '2016-01-01', '2016-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01', '2016-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01',
               ...
               '2010-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2010-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2010-01-01', '2010-01-01'],
              dtype='datetime64[ns]', name='Date', length=652, freq=None)

In [22]:
country_index = bm.index.get_level_values("Country")
country_index

# bm.index.get_level_values(1) # Equivalent as typing "Country". 

# Since this column consists of strings, the index is stored as a regular object index. 

Index(['Argentina', 'Australia', 'Brazil', 'Britain', 'Canada', 'Chile',
       'China', 'Colombia', 'Costa Rica', 'Czech Republic',
       ...
       'Sri Lanka', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey',
       'UAE', 'Ukraine', 'United States', 'Uruguay'],
      dtype='object', name='Country', length=652)

Change Index Level Name with the .set_names() Method

Change one or more names for our index levels in our MultiIndex DataFrame. This method is invoked on the index object and not the original DataFrame. 

In [46]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"], index_col = ["Date", "Country"])
bm.sort_index(inplace = True)
bm.head()

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76
           Britain                   3.67
           Canada                    3.97

In [47]:
# Parameters of the .set_name() method:
# names : The new name(s) you want to change to. 
# level : The old label you want to change. (can provide index label or position).
# inplace : True if you want to overwrite old index object. 

# Changing single index label: Date to Day.
bm.index.set_names(names = "Day", level = "Date", inplace = True)
bm.index

MultiIndex([('2010-01-01',      'Argentina'),
            ('2010-01-01',      'Australia'),
            ('2010-01-01',         'Brazil'),
            ('2010-01-01',        'Britain'),
            ('2010-01-01',         'Canada'),
            ('2010-01-01',          'Chile'),
            ('2010-01-01',          'China'),
            ('2010-01-01',       'Colombia'),
            ('2010-01-01',     'Costa Rica'),
            ('2010-01-01', 'Czech Republic'),
            ...
            ('2016-01-01',    'Switzerland'),
            ('2016-01-01',         'Taiwan'),
            ('2016-01-01',       'Thailand'),
            ('2016-01-01',         'Turkey'),
            ('2016-01-01',            'UAE'),
            ('2016-01-01',        'Ukraine'),
            ('2016-01-01',  'United States'),
            ('2016-01-01',        'Uruguay'),
            ('2016-01-01',      'Venezuela'),
            ('2016-01-01',        'Vietnam')],
           names=['Day', 'Country'], length=652)

In [48]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"], index_col = ["Date", "Country"])
bm.sort_index(inplace = True)
bm.head()

# Change multiple index labels: Date to Day, Country to Location
bm.index.set_names(names = ["Day", "Location"], level = ["Date", "Country"], inplace = True)
bm.index

MultiIndex([('2010-01-01',      'Argentina'),
            ('2010-01-01',      'Australia'),
            ('2010-01-01',         'Brazil'),
            ('2010-01-01',        'Britain'),
            ('2010-01-01',         'Canada'),
            ('2010-01-01',          'Chile'),
            ('2010-01-01',          'China'),
            ('2010-01-01',       'Colombia'),
            ('2010-01-01',     'Costa Rica'),
            ('2010-01-01', 'Czech Republic'),
            ...
            ('2016-01-01',    'Switzerland'),
            ('2016-01-01',         'Taiwan'),
            ('2016-01-01',       'Thailand'),
            ('2016-01-01',         'Turkey'),
            ('2016-01-01',            'UAE'),
            ('2016-01-01',        'Ukraine'),
            ('2016-01-01',  'United States'),
            ('2016-01-01',        'Uruguay'),
            ('2016-01-01',      'Venezuela'),
            ('2016-01-01',        'Vietnam')],
           names=['Day', 'Location'], length=652)

The .sort_index() Method on a MultiIndex DataFrame

A more comprehensive overview. 

In [50]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"], index_col = ["Date", "Country"])
bm.head(3)

Price in US Dollars
Date       Country                       
2016-01-01 Argentina                 2.39
           Australia                 3.74
           Brazil                    3.35

In [54]:
# .sort_index() will by default sort all levels of the MultiIndex starting from the outer layer in ascending order. 
# Date will be sorted first in ascending order, and then, within each Date, the Country level will also be sorted in ascending order. 

# We can sort the MultiIndex in customised ways (not just asc for all index) according to our needs. examples: 

bm.sort_index(level = ["Date", "Country"], ascending = [True, False], inplace = True)
bm.head(3)
# bm.sort_index(ascending = [True, False]) # Equivalent as above. 

#Sorts the Date and Country index in ascending and descending order respectively. 

Price in US Dollars
Date       Country                           
2010-01-01 Uruguay                       3.32
           United States                 3.58
           Ukraine                       1.83

In [58]:
# Sort a single column. 

bm.sort_index(level = "Country", ascending = False, inplace = True) # or level = 1
bm.head(3)

# Sorts by Country in descending order. 

# bm.sort_index(level = "Date") 
# Sorts by Date in ascending order with the Country index left as its original order. 

,,Price in US Dollars
Date,Country,
2016-01-01,Vietnam,2.67
2015-07-01,Vietnam,2.75
2015-01-01,Vietnam,2.81


Extract Rows from a MultiIndex DataFrame

Using the .loc[] and .iloc[] accessors for index labels and positions respectively. 

Rule for .loc[] for MultiIndex DataFrames: 
- First argument is the index label(s). Use a tuple if there are multiple index labels. 
- Second argument is the column name(s). Use a list if there are multiple columns. 

In [69]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"], index_col = ["Date", "Country"])
bm.sort_index(inplace = True)
bm.head(3)

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76

In [75]:
# First, we look at a way that is less ideal. We provide the index labels as the arguments. 

bm.loc["2010-01-01", "Argentina"]
# Returns a series with the original column headers (Price in US Dollars) as the index label in this series and the value for Argentina (1.84). 

# Reason why this is not ideal: There is ambiguity in the second argument of .loc[]. It can either be the second level of the MultiIndex or 
# a column you want to target. In this case, Argentina is not a column but a value in the second level of the MultiIndex. Someone else
# reading your code will be unsure whether Argentina is a index level label or a column name. 

# Providing column name as the second argument. 

bm.loc["2010-01-01", "Price in US Dollars"]

# Nothing is going to break. Returns a MultiIndex series instead. 

Date        Country       
2010-01-01  Argentina         1.84
            Australia         3.98
            Brazil            4.76
            Britain           3.67
            Canada            3.97
            Chile             3.18
            China             1.83
            Colombia          3.91
            Costa Rica        3.52
            Czech Republic    3.71
            Denmark           5.99
            Egypt             2.38
            Euro area         4.84
            Hong Kong         1.91
            Hungary           3.86
            Indonesia         2.24
            Israel            3.99
            Japan             3.50
            Latvia            3.09
            Lithuania         2.87
            Malaysia          2.08
            Mexico            2.50
            New Zealand       3.61
            Norway            7.02
            Pakistan          2.42
            Peru              2.81
            Philippines       2.21
            Poland          

In [80]:
# Best to stick to the traditional approach where the first argument excapsulates all the index label information and the second
# argument encapsulate all the column name information. The structure of this .loc[] accessing is the same as a single index DataFrame. 

# First argument (if there are multiple index labels), can be entered as a TUPLE. It will still be a single input. 
# Second argument is the column name(s). Can be entered as a list. 

bm.loc[("2010-01-01", "Argentina"), "Price in US Dollars"]

Date        Country  
2010-01-01  Argentina    1.84
Name: Price in US Dollars, dtype: float64

In [83]:
# Note on tuples for single values.

bm.loc[("2010-01-01")] 
# Returns DataFrame for 2010-01-01 but includes the Date index column.

bm.loc[("2010-01-01", )]
# Returns single index DataFrame for 2010-01-01 but cuts out the Date Column since all are 2010-01-01. (Preferred way)

,Price in US Dollars
Country,
Argentina,1.84
Australia,3.98
Brazil,4.76
Britain,3.67
Canada,3.97
Chile,3.18
China,1.83
Colombia,3.91
Costa Rica,3.52


In [88]:
# .iloc[] is significantly easier than .loc[]. Because each row is still given an index position in line, rather than having an index position
# within each index label. So, not much has changed from a single index DataFrame for .loc[]. 
bm.iloc[0]
bm.iloc[[10, 20, 50]] # Provide a list if there are multiple rows. 

Price in US Dollars
Date       Country                      
2010-01-01 Denmark                  5.99
           Malaysia                 2.08
2010-07-01 Colombia                 4.39

The .transpose() Method

Swaps the indexes of the DataFrame. Converts the row labels to column headers, and takes the column headers and moves them to become row labels.

.loc[] accessor is used in the same way in terms of arguments. Once again, make sure to provide a value for each index level so Pandas can identify any given entity. 

In [91]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"], index_col = ["Date", "Country"])
bm.sort_index(inplace = True)
bm.head(3)

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76

In [103]:
bm = bm.transpose()
bm

# Returns DataFrame with a single index for our row and for our column we have a MultiIndex, which has two levels, Date and Country. 
# Reassign to bm DataFrame if we want to preserve this transposed DataFrame. 

# A DataFrame can have a column or a row MultiIndex (or both) and that gives us a lot of flexibility in how we organise the headers 
# for our data. We can have categories and subcategories in our columns or groupings and subgroupings within it. That allows for 
# very powerful quick slicing for large datasets. 

Date                2010-01-01                                              \
Country              Argentina Australia Brazil Britain Canada Chile China   
Price in US Dollars       1.84      3.98   4.76    3.67   3.97  3.18  1.83   

Date                                                    ...  2016-01-01  \
Country             Colombia Costa Rica Czech Republic  ... Switzerland   
Price in US Dollars     3.91       3.52           3.71  ...        6.44   

Date                                                                    \
Country             Taiwan Thailand Turkey   UAE Ukraine United States   
Price in US Dollars   2.08     3.09   3.41  3.54    1.54          4.93   

Date                                           
Country             Uruguay Venezuela Vietnam  
Price in US Dollars    3.74      0.66    2.67  

[1 rows x 652 columns]

In [104]:
# Slicing using the .loc[] accessor. Will be exactly the same; first argument represents the row labels and the second argument will 
# be the column index level values. 

bm.loc[("Price in US Dollars", )]

# Returns a MultiIndex series consisting of the column headers pulled into a MultiIndex row index. 

Date        Country      
2010-01-01  Argentina        1.84
            Australia        3.98
            Brazil           4.76
            Britain          3.67
            Canada           3.97
                             ... 
2016-01-01  Ukraine          1.54
            United States    4.93
            Uruguay          3.74
            Venezuela        0.66
            Vietnam          2.67
Name: Price in US Dollars, Length: 652, dtype: float64

In [106]:
# We can provide a second argument. 

bm.loc[("Price in US Dollars", ), ("2010-01-01", )] 

# Returns a subset of the countries that fall under the index level value of 2010-01-01 for our Date level. 

Date                2010-01-01                                              \
Country              Argentina Australia Brazil Britain Canada Chile China   
Price in US Dollars       1.84      3.98   4.76    3.67   3.97  3.18  1.83   

Date                                                    ...                   \
Country             Colombia Costa Rica Czech Republic  ... Sri Lanka Sweden   
Price in US Dollars     3.91       3.52           3.71  ...      1.83   5.51   

Date                                                                  \
Country             Switzerland Taiwan Thailand Turkey   UAE Ukraine   
Price in US Dollars         6.3   2.36     2.11   3.83  2.99    1.83   

Date                                       
Country             United States Uruguay  
Price in US Dollars          3.58    3.32  

[1 rows x 43 columns]

In [107]:
bm.loc[("Price in US Dollars", ), ("2010-01-01", "Sri Lanka")]

Date,2010-01-01
Country,Sri Lanka
Price in US Dollars,1.83


In [123]:
# Let's say we want to pull out the columns of Sri Lanka through Ukraine for those countries that sit under 2010-01-01. 

bm.loc[("Price in US Dollars", ), ("2010-01-01", ("Sri Lanka", "Ukraine"))]
# This returns only the values for Sri Lanka and Ukraine. 

bm.loc[("Price in US Dollars", ), ("2010-01-01", "Sri Lanka"): ("2010-01-01", "Ukraine")]
# Returns desired result. 

Date                2010-01-01                                            \
Country              Sri Lanka Sweden Switzerland Taiwan Thailand Turkey   
Price in US Dollars       1.83   5.51         6.3   2.36     2.11   3.83   

Date                               
Country               UAE Ukraine  
Price in US Dollars  2.99    1.83

In [125]:
# Uncomment this cell to see what the transposed result looks like. 

# bm = bm.transpose()

# bm.loc[("2010-01-01", "Sri Lanka"): ("2010-01-01", "Ukraine"), ("Price in US Dollars", )]

Price in US Dollars
Date       Country                         
2010-01-01 Sri Lanka                   1.83
           Sweden                      5.51
           Switzerland                 6.30
           Taiwan                      2.36
           Thailand                    2.11
           Turkey                      3.83
           UAE                         2.99
           Ukraine                     1.83

The .swaplevel() Method

Allows us to swap or move around two index levels within our MultiIndex. 

In [127]:
bm = pd.read_csv("bigmac.csv", parse_dates = ["Date"], index_col = ["Date", "Country"])
bm.sort_index(inplace = True)
bm.head(3)

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76

In [150]:
# Swap the Date and Country index levels.

# Parameters: i and j representing the two indexes you want to swap (if there are > two levels to choose from). Order doesn't matter.

bm = bm.swaplevel() # Since there's only two index levels in this case, we can leave the arguments blank. 
bm

Price in US Dollars
Date       Country                           
2010-01-01 Argentina                     1.84
           Australia                     3.98
           Brazil                        4.76
           Britain                       3.67
           Canada                        3.97
...                                       ...
2016-01-01 Ukraine                       1.54
           United States                 4.93
           Uruguay                       3.74
           Venezuela                     0.66
           Vietnam                       2.67

[652 rows x 1 columns]

The .stack() Method

Takes the columns at the top (column axis) and moves that to the main index (on the left , the horizontal one).

In [151]:
world = pd.read_csv("worldstats.csv", index_col = ["country", "year"])
world.head(3)

Population           GDP
country    year                           
Arab World 2015  392022276.0  2.530102e+12
           2014  384222592.0  2.873600e+12
           2013  376504253.0  2.846994e+12

In [162]:
# Move the column indexes to the main index. 
world.stack() # Moves all columns if left blank.

# Returns a series (single column) with three levels of index. 
# So in order to access a specific value, we will have to provide the information for the country, year and the statistic (population or gdp). 

country     year            
Arab World  2015  Population    3.920223e+08
                  GDP           2.530102e+12
            2014  Population    3.842226e+08
                  GDP           2.873600e+12
            2013  Population    3.765043e+08
                                    ...     
Zimbabwe    1962  GDP           1.117602e+09
            1961  Population    3.876638e+06
                  GDP           1.096647e+09
            1960  Population    3.752390e+06
                  GDP           1.052990e+09
Length: 22422, dtype: float64

In [164]:
# .to_frame() method converts a series to a DataFrame. 

world.stack().to_frame()

0
country    year                         
Arab World 2015 Population  3.920223e+08
                GDP         2.530102e+12
           2014 Population  3.842226e+08
                GDP         2.873600e+12
           2013 Population  3.765043e+08
...                                  ...
Zimbabwe   1962 GDP         1.117602e+09
           1961 Population  3.876638e+06
                GDP         1.096647e+09
           1960 Population  3.752390e+06
                GDP         1.052990e+09

[22422 rows x 1 columns]

The .unstack() Method - Part 1 of 3

Does the reverse of the .stack() method. Takes an outer level of a MultiIndex and moves it to serve as a column. 

In [165]:
world = pd.read_csv("worldstats.csv", index_col = ["country" , "year"])

In [169]:
s = world.stack() # Returns a 3 level MultiIndex
s.head(3)

country     year            
Arab World  2015  Population    3.920223e+08
                  GDP           2.530102e+12
            2014  Population    3.842226e+08
dtype: float64

In [175]:
# .unstack() method does the reverse. 

s.unstack()
# Takes the innermost level (Population and GDP) and moves it to the column. Returns the original  DataFrame. 

# We can method chain the .unstack() method to unstack (or move) the next index level to become a column. 
s.unstack().unstack()
# Now, we have a MultiIndex in the columns section. 

# Call it again to return a series which is the reverse of the series in the previous lesson. 
s.unstack().unstack().unstack()

            year  country           
Population  1960  Afghanistan           8.994793e+06
                  Albania                        NaN
                  Algeria               1.112489e+07
                  Andorra                        NaN
                  Angola                         NaN
                                            ...     
GDP         2015  West Bank and Gaza    1.267740e+10
                  World                 7.343364e+13
                  Yemen, Rep.                    NaN
                  Zambia                2.120156e+10
                  Zimbabwe              1.389294e+10
Length: 28224, dtype: float64

 The .unstack() Method - Part 2 of 3
 
 Introduction to a wide selection of arguments that can be provided to this method to affect which of the indexes of the MultiIndex will be moved to the column axis. 

In [174]:
world = pd.read_csv("worldstats.csv", index_col = ["country", "year"])
world.head(3)

Population           GDP
country    year                           
Arab World 2015  392022276.0  2.530102e+12
           2014  384222592.0  2.873600e+12
           2013  376504253.0  2.846994e+12

In [177]:
s = world.stack()
s.head(3)

country     year            
Arab World  2015  Population    3.920223e+08
                  GDP           2.530102e+12
            2014  Population    3.842226e+08
dtype: float64

In [180]:
# For .unstack(), the default process is to unstack from the innermost level. But there's parameters to make it more specific. 
# Parameters:
# level : specifies the level you want to move to the column index. (can use index position)
# fill_value : specifies what you want to fill the NULL values with. 

s.unstack(level = "country")
# Moved the country level to the column index. 

s.unstack(level = -1) 
# Also takes index positions. Here, we start counting from the last index, this targets the last index (Population and GDP level). 

Population           GDP
country     year                          
Afghanistan 1960   8994793.0  5.377778e+08
            1961   9164945.0  5.488889e+08
            1962   9343772.0  5.466667e+08
            1963   9531555.0  7.511112e+08
            1964   9728645.0  8.000000e+08
...                      ...           ...
Zimbabwe    2011  14255592.0  1.095623e+10
            2012  14565482.0  1.239272e+10
            2013  14898092.0  1.349023e+10
            2014  15245855.0  1.419691e+10
            2015  15602751.0  1.389294e+10

[11211 rows x 2 columns]

The .unstack() Method - Part 3 of 3 

How to unstack multiple levels, basically migrating more than one levels of the MultiIndex to the column index. 

In [181]:
world = pd.read_csv("worldstats.csv", index_col = ["country", "year"])
s = world.stack() # Returns series with 3 levels of row index. 
s.head(3)

country     year            
Arab World  2015  Population    3.920223e+08
                  GDP           2.530102e+12
            2014  Population    3.842226e+08
dtype: float64

In [183]:
s.unstack(level = ["year", "country"])

# Moves the year to become the first level of the column index and then moves country to become the second level. 

year,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
country,Arab World,Arab World,Arab World,Arab World,Arab World,Arab World,Arab World,Arab World,Arab World,Arab World,...,Zimbabwe,Zimbabwe,Zimbabwe,Zimbabwe,Zimbabwe,Zimbabwe,Zimbabwe,Zimbabwe,Zimbabwe,Zimbabwe
Population,3.920223e+08,3.842226e+08,3.765043e+08,3.688026e+08,3.610318e+08,3.531122e+08,3.450542e+08,3.368865e+08,3.287666e+08,3.209067e+08,...,5.036321e+06,4.874113e+06,4.718612e+06,4.568320e+06,4.422132e+06,4.279561e+06,4.140804e+06,4.006262e+06,3.876638e+06,3.752390e+06
GDP,2.530102e+12,2.873600e+12,2.846994e+12,2.773270e+12,2.497945e+12,2.103825e+12,1.798878e+12,2.081343e+12,1.641666e+12,1.404190e+12,...,1.747999e+09,1.479600e+09,1.397002e+09,1.281750e+09,1.311436e+09,1.217138e+09,1.159512e+09,1.117602e+09,1.096647e+09,1.052990e+09


In [185]:
s.unstack(level = ["country", "year"])

# Opoosite order than previous cell. 

country       Arab World                                            \
year                2015          2014          2013          2012   
Population  3.920223e+08  3.842226e+08  3.765043e+08  3.688026e+08   
GDP         2.530102e+12  2.873600e+12  2.846994e+12  2.773270e+12   

country                                                             \
year                2011          2010          2009          2008   
Population  3.610318e+08  3.531122e+08  3.450542e+08  3.368865e+08   
GDP         2.497945e+12  2.103825e+12  1.798878e+12  2.081343e+12   

country                                 ...      Zimbabwe                \
year                2007          2006  ...          1969          1968   
Population  3.287666e+08  3.209067e+08  ...  5.036321e+06  4.874113e+06   
GDP         1.641666e+12  1.404190e+12  ...  1.747999e+09  1.479600e+09   

country                                                             \
year                1967          1966          1965          1964   
Population  4.718612e+06  4.568320e+06  4.422132e+06  4.279561e+06   
GDP         1.397002e+09  1.281750e+09  1.311436e+09  1.217138e+09   

country                                                             
year                1963          1962          1961          1960  
Population  4.140804e+06  4.006262e+06  3.876638e+06  3.752390e+06  
GDP         1.159512e+09  1.117602e+09  1.096647e+09  1.052990e+09  

[2 rows x 11211 columns]

In [186]:
# There are NULL values for missing values in the DataFrame to maintain the DataFrames structure/composition.
# fill_value : parameter to specify what you want to populate the NULL values with. 

s.unstack(level = "year", fill_value = "Missing")

year                           1960         1961         1962         1963  \
country                                                                      
Afghanistan Population  8.99479e+06  9.16494e+06  9.34377e+06  9.53156e+06   
            GDP         5.37778e+08  5.48889e+08  5.46667e+08  7.51111e+08   
Albania     Population      Missing      Missing      Missing      Missing   
            GDP             Missing      Missing      Missing      Missing   
Algeria     Population  1.11249e+07  1.14049e+07  1.16902e+07  1.19851e+07   
...                             ...          ...          ...          ...   
Yemen, Rep. GDP             Missing      Missing      Missing      Missing   
Zambia      Population  3.04959e+06  3.14285e+06  3.24066e+06  3.34289e+06   
            GDP          6.9874e+08   6.8236e+08   6.7928e+08   7.0434e+08   
Zimbabwe    Population  3.75239e+06  3.87664e+06  4.00626e+06   4.1408e+06   
            GDP         1.05299e+09  1.09665e+09   1.1176e+09  1.15951e+09   

year                           1964         1965         1966         1967  \
country                                                                      
Afghanistan Population  9.72864e+06  9.93536e+06  1.01488e+07  1.03686e+07   
            GDP               8e+08  1.00667e+09      1.4e+09  1.67333e+09   
Albania     Population      Missing      Missing      Missing      Missing   
            GDP             Missing      Missing      Missing      Missing   
Algeria     Population   1.2296e+07   1.2627e+07  1.29803e+07  1.33542e+07   
...                             ...          ...          ...          ...   
Yemen, Rep. GDP             Missing      Missing      Missing      Missing   
Zambia      Population  3.44927e+06  3.55969e+06  3.67409e+06  3.79286e+06   
            GDP          8.2264e+08   1.0612e+09    1.239e+09  1.34064e+09   
Zimbabwe    Population  4.27956e+06  4.42213e+06  4.56832e+06  4.71861e+06   
            GDP         1.21714e+09  1.31144e+09  1.28175e+09    1.397e+09   

year                           1968         1969  ...         2006  \
country                                           ...                
Afghanistan Population  1.05998e+07  1.08495e+07  ...  2.51836e+07   
            GDP         1.37333e+09  1.40889e+09  ...   7.0576e+09   
Albania     Population      Missing      Missing  ...  2.99255e+06   
            GDP             Missing      Missing  ...  8.99264e+09   
Algeria     Population  1.37444e+07  1.41444e+07  ...  3.37493e+07   
...                             ...          ...  ...          ...   
Yemen, Rep. GDP             Missing      Missing  ...  1.90817e+10   
Zambia      Population  3.91693e+06  4.04748e+06  ...  1.23815e+07   
            GDP         1.57374e+09   1.9264e+09  ...  1.27569e+10   
Zimbabwe    Population  4.87411e+06  5.03632e+06  ...  1.31279e+07   
            GDP          1.4796e+09    1.748e+09  ...   5.4439e+09   

year                           2007         2008         2009         2010  \
country                                                                      
Afghanistan Population  2.58775e+07  2.65287e+07  2.72073e+07  2.79622e+07   
            GDP         9.84384e+09  1.01905e+10  1.24869e+10  1.59368e+10   
Albania     Population  2.97002e+06  2.94731e+06  2.92752e+06  2.91302e+06   
            GDP          1.0701e+10  1.28814e+10  1.20442e+10   1.1927e+10   
Algeria     Population   3.4262e+07  3.48111e+07  3.54018e+07  3.60362e+07   
...                             ...          ...          ...          ...   
Yemen, Rep. GDP         2.56337e+10  3.03972e+10  2.84595e+10  3.09068e+10   
Zambia      Population  1.27387e+07  1.31146e+07  1.35078e+07  1.39174e+07   
            GDP          1.4057e+10  1.79109e+10  1.53283e+10  2.02656e+10   
Zimbabwe    Population  1.32978e+07  1.34955e+07   1.3721e+07  1.39739e+07   
            GDP         5.29195e+09   4.4157e+09  8.15708e+09  9.42216e+09   

year                           2011         

The .pivot() Method

Used to re-orient a dataset. Takes the values that are currently in a column and converts them to column headers. 

In [193]:
sm = pd.read_csv("salesmen.csv", parse_dates = ["Date"])
sm["Salesman"] = sm["Salesman"].astype("category")
sm.head(3)

,Date,Salesman,Revenue
0,2016-01-01,Bob,7172
1,2016-01-02,Bob,6362
2,2016-01-03,Bob,5982


In [195]:
sm["Salesman"].value_counts()
# We can deduce that the DataFrame recorded the sales data for every salesmen for every day of a year. 
# Or how much revenue each salesmen generated on a day to day basis. 
# Convert Salesman column to category due to small number of unique values. 

# There are 366 rows for every salesman, which is not the ideal format to store this data. 
# We can pivot this data to something a lot more readable and something that saves more space. 

Ronald    366
Oscar     366
Jeb       366
Dave      366
Bob       366
Name: Salesman, dtype: int64

In [196]:
# .pivot() paramters. 
# index : column(s) to use to make the new frame's index.
# columns column(s) to use to make the new frame's column(s).
# values : column(s) to use to populate the new frame's values. 

sm.pivot(index = "Date", columns = "Salesman", values = "Revenue")

Salesman,Bob,Dave,Jeb,Oscar,Ronald
Date,,,,,
2016-01-01,7172,1864,4430,5250,2639
2016-01-02,6362,8278,8026,8661,4951
2016-01-03,5982,4226,5188,7075,2703
2016-01-04,7917,3868,3144,2524,4258
2016-01-05,7837,2287,938,2793,7771
...,...,...,...,...,...
2016-12-27,2045,2843,6666,835,2981
2016-12-28,100,8888,1243,3073,6129
2016-12-29,4115,9490,3498,6424,7662


The .pivot_table() Method

Operates as it does in Excel. Ideal for taking a DataFrame and aggregating it to look the values as a whole. For example you want to take the sum based on values in a certain column, or the mean and you want to create groupings or sections based on common values. 

In [216]:
# Import and perform optimzations of new dataset. 

foods = pd.read_csv("foods.csv")

# Check info on DataFrame

# foods.dtypes
# foods["City"].value_counts()
# foods["Frequency"].value_counts()
# foods["Item"].value_counts()

# We can convert some columns to the category dtype. 
foods["Gender"] = foods["Gender"].astype("category")
foods["City"] = foods["City"].astype("category")
foods["Frequency"] = foods["Frequency"].astype("category")
foods["Item"] = foods["Item"].astype("category")
foods.dtypes
foods.head(3)

,First Name,Gender,City,Frequency,Item,Spend
0,Wanda,Female,Stamford,Weekly,Burger,15.66
1,Eric,Male,Stamford,Daily,Chalupa,10.56
2,Charles,Male,New York,Never,Sushi,42.14


In [226]:
# Example: Get a result where the index is going to be the Gender and we want to aggregate the total spend for each gender. 

foods.pivot_table(values = "Spend", index = "Gender", aggfunc = "sum")
# Aggregate total value for Spend for each gender (set as index). 

# Aggregate columns with numeric values

# Loops through the Gender column and finds all the unique values (Male and Female) and then take the values in the Spend column and 
# performs a specified operation on them. 


,Spend
Gender,
Female,25963.33
Male,24106.04


In [228]:
foods.pivot_table(values = "Spend", columns = "Gender", aggfunc = "mean")
# Aggregate mean value for Spend for each gender (set as index).

Gender,Female,Male
Spend,50.709629,49.397623


In [231]:
foods.pivot_table(values = "Spend", columns = "Item", aggfunc = "sum")
# Sum of sales by the item. 

Item,Burger,Burrito,Chalupa,Donut,Ice Cream,Sushi
Spend,7765.73,8270.44,7644.52,8758.76,8886.99,8742.93


In [235]:
# Can create a MultiIndex series.

foods.pivot_table(values = "Spend", index   = ["Gender", "Item"], columns = ["City"], aggfunc = "sum")

City              New York  Philadelphia  Stamford
Gender Item                                       
Female Burger      1239.04       1639.24   1216.02
       Burrito      978.95       1458.76   1820.11
       Chalupa      876.58       1673.33   1602.35
       Donut       1446.78       1639.26   1656.96
       Ice Cream   1521.62       1479.22   1032.03
       Sushi       1480.29       1742.88   1459.91
Male   Burger      1294.09        938.18   1439.16
       Burrito     1399.40       1312.93   1300.29
       Chalupa     1227.77       1114.23   1150.26
       Donut       1345.27       1249.36   1421.13
       Ice Cream   1603.63       2191.27   1059.22
       Sushi       1396.15       1395.88   1267.82

In [237]:
# Example of further breakdown of the data. 

foods.pivot_table(values = "Spend", index = ["Gender", "Item"], columns = ["Frequency", "City"], aggfunc = "sum")

# Looking at the first value 262.67: represents the total spend by female customers who bought  burgers from New York on a daily basis. 

Frequency           Daily                        Monthly               \
City             New York Philadelphia Stamford New York Philadelphia   
Gender Item                                                             
Female Burger      262.67       231.68   144.66   171.86       215.05   
       Burrito     224.45       321.57   195.63   122.74        34.28   
       Chalupa      43.19        23.49    95.70   158.37       289.96   
       Donut       478.10       247.40   124.35   284.53        50.25   
       Ice Cream   262.19       177.69    92.88    92.53        74.51   
       Sushi        81.07       348.53   393.27    93.16        78.71   
Male   Burger      319.46       112.70   197.72   187.29       213.14   
       Burrito     236.21       165.76   276.23   147.54        89.58   
       Chalupa      54.09       274.81   192.64   333.76        90.70   
       Donut       230.00       190.71   129.42   183.73       259.29   
       Ice Cream   178.22       148.69    15.17   152.20       386.55   
       Sushi       225.57       280.93      NaN   430.86       155.59   

Frequency                    Never                          Often  ...  \
City             Stamford New York Philadelphia Stamford New York  ...   
Gender Item                                                        ...   
Female Burger      238.49    97.89       218.97    90.97    94.96  ...   
       Burrito      67.94   189.73       382.30   366.34   103.60  ...   
       Chalupa     161.98    35.15       121.97   156.36    39.73  ...   
       Donut       229.30    56.07       433.58   157.33   130.63  ...   
       Ice Cream   125.85   206.15       156.03    77.66   232.26  ...   
       Sushi       108.39    69.33       190.58    19.56   343.94  ...   
Male   Burger       27.16    90.32        17.31      NaN    55.47  ...   
       Burrito     119.60    86.78       378.32   351.84   189.92  ...   
       Chalupa     343.76   199.09       242.98   138.70   251.52  ...   
       Donut       119.53   219.63        53.65   219.64   139.80  ...   
       Ice Cream    74.36    57.54       321.94   119.77   347.55  ...   
       Sushi        93.03   164.01       245.77   204.90     9.31  ...   

Frequency            Once   Seldom                         Weekly  \
City             Stamford New York Philadelphia Stamford New York   
Gender Item                                                         
Female Burger       95.05    63.16       409.05    97.53   184.35   
       Burrito     168.01   167.54       198.11   234.49    39.69   
       Chalupa      40.59   160.00       274.51   175.25   171.52   
       Donut       158.24    90.81       183.25   104.66   285.56   
       Ice Cream   279.33   242.35       203.10   117.73   227.62   
       Sushi       194.75    87.70       314.24    27.82   410.89   
Male   Burger      326.64   225.68       188.06   159.75    69.69   
       Burrito      30.15   202.40        27.71     9.84   256.74   
       Chalupa        NaN    11.69       130.75   174.02   163.20   
       Donut       139.89    16.25        99.01   163.31    74.44   
       Ice Cream   182.71   270.42       138.44    40.83   223.86   
       Sushi       314.65   160.49        91.26   224.15   150.15   

Frequency                                Yearly                        
City             Philadelphia Stamford New York Philadelphia Stamford  
Gender Item                                                            
Female Burger           32.00   155.02   259.30       123.17   307.03  
       Burrito          62.82   230.91    71.26       116.75   172.98  
       Chalupa          84.41   204.69   157.82       504.44   348.16  
       Donut           208.80   440.06    62.95       175.23   224.48  
       Ice Cream       285.28   125.03   151.67       159.86    15.24  
       Sushi           334.00   315.36   232.41       145.85   362.74  
Male   Burger           99.89   310.21    49.61       197.36   225.07  
       Burrito         289.25   161.8

In [ ]:
# Additional aggregation functions. 
# sum ,mean, count (number of rows that fall into that criteria), min, max, 

In [242]:
# Another way to apply .pivot_table() method is directly through pd. 

foods.pivot_table(values = "Spend", index = "Gender", columns = "Item", aggfunc = "sum")

pd.pivot_table(data = foods, values = "Spend", index = "Gender", columns = "Item", aggfunc = "sum")

# Both of these yields the same results. 

Item,Burger,Burrito,Chalupa,Donut,Ice Cream,Sushi
Gender,,,,,,
Female,4094.30,4257.82,4152.26,4743.00,4032.87,4683.08
Male,3671.43,4012.62,3492.26,4015.76,4854.12,4059.85


The pd.melt() Method 

Esentially the reverse operation as the .pivot_table(). Reorganizes an aggregated dataset and converts it into a tabular format. 

In [243]:
sales = pd.read_csv("quarters.csv")
sales

# DataFrame with ten salesmen and how much they made in each quarter.
# Use pd.melt() to convert this into a more tabular structure. 

,Salesman,Q1,Q2,Q3,Q4
0,Boris,602908,233879,354479,32704
1,Bob,43790,514863,297151,544493
2,Tommy,392668,113579,430882,247231
3,Travis,834663,266785,749238,570524
4,Donald,580935,411379,110390,651572
5,Ted,656644,70803,375948,321388
6,Jeb,486141,600753,742716,404995
7,Stacy,479662,742806,770712,2501
8,Morgan,992673,879183,37945,293710


In [251]:
pd.melt(sales, id_vars = "Salesman", var_name = "Quarter", value_name = "Revenue")

# id_vars : represents the column that will be preserved (kept in place). 

# Essentially, we took all the columns and condensed them to a single variables (Quarter) column. 
# Each value has it's own row. Four rows for every salesman. One row for each quarter for every single salesman. 

,Salesman,Quarter,Revenue
0,Boris,Q1,602908
1,Bob,Q1,43790
2,Tommy,Q1,392668
3,Travis,Q1,834663
4,Donald,Q1,580935
5,Ted,Q1,656644
6,Jeb,Q1,486141
7,Stacy,Q1,479662
8,Morgan,Q1,992673
9,Boris,Q2,233879
